In [7]:
#import some libraries
import os
import cv2
import numpy as np
import tensorflow as tf
import keras

## Using TensorFlow like Numpy
### A tensor is usally a multidimensonal array, but it can also hold a scalar (vô hướng) (a simple value, such as 42). These tensors will be important when we create custom cost functions, custom metrics, custom layers, ...

In [2]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) #Matrix  

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

**Notes**: These value above show us the shape of matrix with 2 rows and 3 columns (shape =(rows, columns)), and data types=float32 as well as the numpy version

In [4]:
#Indexing works much like in Numpy
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t[0, :]) #[1. 2. 3.]
print(t[:, 1:]) #[[2. 3.], [5. 6.]] (columns 1 and 2)


tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)


**Notes**: We can see the difference betweem two shape of array. With the shape of (3,), it is a 1-ranked array not a dimensional array (e.g shape=(3,1))

In [5]:
# All sorts of tensor operations are available:
print(t + 10) #add 10 to each element
print(tf.square(t)) #square each element
print(tf.reduce_sum(t)) #sum all elements
print(tf.reduce_sum(t, 0)) #sum each column
print(tf.reduce_sum(t, 1)) #sum each row
print(tf.reduce_mean(t)) #mean of all elements
print(tf.reduce_mean(t, 0)) #mean of each column
print(tf.reduce_mean(t, 1)) #mean of each row
print(tf.transpose(t)) #swap rows and columns

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(21.0, shape=(), dtype=float32)
tf.Tensor([5. 7. 9.], shape=(3,), dtype=float32)
tf.Tensor([ 6. 15.], shape=(2,), dtype=float32)
tf.Tensor(3.5, shape=(), dtype=float32)
tf.Tensor([2.5 3.5 4.5], shape=(3,), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor(
[[1. 4.]
 [2. 5.]
 [3. 6.]], shape=(3, 2), dtype=float32)


## Tensors and Numpy
### You can create a tensor from a NumPy array, and vice versa

In [6]:
a = np.array([2., 4., 5.])
print(tf.constant(a)) #[2. 4. 5.]
print(t.numpy()) #[1. 2. 3.]

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
[[1. 2. 3.]
 [4. 5. 6.]]


In [7]:
print(tf.square(a)) #[4. 16. 25.]
print(np.square(t)) #[[1. 4. 9.], [16. 25. 36.]]

tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)
[[ 1.  4.  9.]
 [16. 25. 36.]]


**Warning**: Numpy uses 64-bit precision by default, while TensorFlow uses 32-bit precision (more than enough for neural networks, plus it faster and uses less RAM). So when we create a tensor from a NumPy array, make sure to set dtype=tf.float32

## Type conversions

In [8]:
# Using tf.cast() to convert between types
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32) #42

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## Variables
### The tf.Tensor values we have seen so far ar immutable (you can not modify them). This means that we cannot use regular tensors to implement weights in a neural network, since they need to be tweaked by backpropagation. That's why we need tf.Variable

In [9]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [10]:
"""
A Tf.Variable acts much like a tf.Tensor: you can perform the same operation with it, it plays nicely with NumPy as well.
But it can also be modified in place using tf.assign().
"""
v.assign(2 *  v)    # v is now [[2., 4., 6.], [8., 10., 12.]]
v[0,1].assign(42.)  # v is now [[2., 42., 6.], [8., 10., 12.]]
v[:, 2].assign([0., 1.])    # v is now [[2., 42., 0.], [8., 10., 1.]]
v.scatter_nd_update([[0, 0], [1, 2]], updates=[100., 200.]) # v is now [[100., 42., 0.], [8., 10., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## Other Data Structures

In [5]:
#Sparse tensors
#A sparse tensor is a tensor that has a very small number of elements, and most of them are zero.
t = tf.SparseTensor([[0, 0], [1, 2]], [100., 200.], [3, 3])
print(t) #SparseTensor(indices=[[0 0], [1 2]], values=[100. 200.], dense_shape=[3 3])

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([100. 200.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([3 3], shape=(2,), dtype=int64))


In [11]:
#Tensor Arrays
#A tensor array is a tensor that can be dynamically resized.
ta = tf.TensorArray(tf.float32, size=3)
ta = ta.write(0, [[1., 2.], [3., 4.]])
print(ta.read) #TensorArray(size=3, dtype=float32, dynamic_size=False, clear_after_read=True)

<bound method TensorArray.read of <tensorflow.python.ops.tensor_array_ops.TensorArray object at 0x0000022D86020488>>


In [17]:
#Ragged Tensor
#A ragged tensor is a tensor that has a variable number of dimensions.
rt = tf.RaggedTensor.from_row_splits(values=[1, 2, 3, 4, 5, 6], row_splits=[0, 2, 3, 5, 6])
print(rt) #RaggedTensor(values=Tensor(values, dtype=int32), row_splits=Tensor(row_splits, dtype=int64))



<tf.RaggedTensor [[1, 2], [3], [4, 5], [6]]>


In [18]:
#String Tensors
"""
A string tensor are regular tensors of type tf.string.
These represent byte strings, not Unicode strings. Alternatively, you can respresent Unicode strings a using tf.int32, where each item represents a Unicode code point (a 32-bit integer). 
"""
st = tf.constant(["Hello", "World"])
print(st) #Tensor("Const:0", shape=(2,), dtype=string)

tf.Tensor([b'Hello' b'World'], shape=(2,), dtype=string)


In [23]:
# Queues
"""
Store tensors across multiple steps in a computation.
First In, First Out (FIFO) queues.
"""
q = tf.queue.FIFOQueue(3, "float")
print(q.shapes) #FIFOQueue(3, dtype=float32, shapes=None, shared_name=None, name=None)

[TensorShape(None)]


# Customizing Models and Training Algorithms

## Custom Loss Function
Suppose you start by trying to clean up your dataset by removing or fixing the outliers, but that turns out to be unsufficient; the dataset is still noisy.Which loss function should you use?

    1. Mean Squared Error (MSE) --> might penalize large errors to much and cause the model to be imprecise
    
    2. Mean Absolute Error (MAE) --> would not penalize outliers as much, but training might take a while to converge, and the trained model might not be very precise

    3. Huber Loss --> a combination of MSE and MAE, is quadratic when the error is smaller than a threshold but linear when the error is larger than threshold. The linear part makes it less sensitive to outliers than the MSE, and the quadratic part allows it to converge faster and be more precise than the MAE

In [6]:
#Huber loss 
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

t = create_huber()
t(1., 2.)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

**Notes**: For better performance, you should use a vectorized implementation.

In [ ]:
"""
When you save the model, the threshold will not be saved.
You have to set it again when loading the model.
To solve this problem, you create a subclass of the keras.losses.Loss class and then implementing its get_config() method.
"""

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        super(HuberLoss, self).__init__(**kwargs)
        self.threshold = threshold
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        config = {'threshold': self.threshold}
        base_config = super(HuberLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))